In [1]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')

In [2]:
import torch

In [3]:
device = 0 if torch.cuda.is_available() else -1
device

-1

In [4]:
books["categories"].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Aged women,1
475,Imperialism,1
476,Human-animal relationships,1
477,Amish,1


In [5]:
books["categories"].value_counts().reset_index().query("count>=50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [6]:
category_mapping = {'Fiction' : "Fiction",
'Juvenile Fiction': "Children's Fiction",
'Biography & Autobiography': "Nonfiction",
'History': "Nonfiction",
'Literary Criticism': "Nonfiction",
'Philosophy': "Nonfiction",
'Religion': "Nonfiction",
'Comics & Graphic Novels': "Fiction",
'Drama': "Fiction",
'Juvenile Nonfiction': "Children's Nonfiction",
'Science': "Nonfiction",
'Poetry': "Fiction",
"Literary Collections": "Fiction",} 

In [7]:
books["simple_category"] = books["categories"].map(category_mapping)

In [9]:
books[~(books["simple_category"].isna())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title and subtitle,tagged_description,simple_category
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's Fiction
31,9780007105045,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045 ""The two works 'On fairy-stories...",Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5188,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5189,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [10]:
from transformers import pipeline

fiction_categories = ["Fiction","Nonfiction"]

pipe = pipeline("zero-shot-classification", 
                model="facebook/bart-large-mnli",
                device=0 if torch.cuda.is_available() else -1)

d:\Python programing\Book_Recommender_System\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [11]:
sequence = books.loc[books["simple_category"] == "Fiction","description"].reset_index(drop=True)[0]


In [12]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [13]:
import numpy as np

max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_labels = pipe(sequence, fiction_categories)["labels"][max_index]
max_labels

'Fiction'

In [14]:
from tqdm import tqdm

In [15]:
# Filter descriptions once
fiction_descriptions = books.loc[books["simple_category"]=="Fiction", "description"].reset_index(drop=True)[:300]
nonfiction_descriptions = books.loc[books["simple_category"]=="Nonfiction", "description"].reset_index(drop=True)[:300]


In [16]:
def generate_predictions(sequences, categories, batch_size=16):
    """
    Predicts top categories for a list of sequences using zero-shot classification.
    """
    predicted_cats = []
    
    for i in tqdm(range(0, len(sequences), batch_size)):
        batch_sequences = sequences[i:i+batch_size].tolist()
        results = pipe(batch_sequences, candidate_labels=categories, batch_size=batch_size)
        for result in results:
            max_index = np.argmax(result["scores"])
            predicted_cats.append(result["labels"][max_index])
    
    return predicted_cats


In [17]:
# Predict Fiction and Nonfiction in batches
predicted_fiction = generate_predictions(fiction_descriptions, fiction_categories)
predicted_nonfiction = generate_predictions(nonfiction_descriptions, fiction_categories)

# Combine results
predicted_cats = predicted_fiction + predicted_nonfiction
actual_cats = ["Fiction"] * len(fiction_descriptions) + ["Nonfiction"] * len(nonfiction_descriptions)


100%|██████████| 19/19 [06:23<00:00, 20.19s/it]


In [18]:
df_predictions = pd.DataFrame({
    "description": list(fiction_descriptions) + list(nonfiction_descriptions),
    "predicted_category": predicted_cats,
    "actual_category": actual_cats
})

df_predictions.head(10)

,description,predicted_category,actual_category
0,A NOVEL THAT READERS and critics have been eag...,Fiction,Fiction
1,"A memorable, mesmerizing heroine Jennifer -- b...",Fiction,Fiction
2,"Tricked once more by his wily half-brother, Ly...",Fiction,Fiction
3,"""The two works 'On fairy-stories' and 'Leaf by...",Nonfiction,Fiction
4,A Few Weeks After Marrying An Attractive Young...,Nonfiction,Fiction
5,"For sixty years, Jewish refugees and their des...",Fiction,Fiction
6,'The Harper's Essay' is reprinted in this volu...,Nonfiction,Fiction
7,Designed to take fans of The Hobbit and The Lo...,Fiction,Fiction
8,"Miller's groundbreaking first novel, banned in...",Fiction,Fiction
9,A retelling of The Mabinogion in novel form. M...,Fiction,Fiction


In [19]:
df_predictions["correct_predictions"] = np.where(
    df_predictions["actual_category"] == df_predictions["predicted_category"], 1, 0
)

In [20]:
df_predictions["correct_predictions"].sum()/len(df_predictions)

np.float64(0.7683333333333333)

In [21]:
isbn = []
predicted_cats = []

missing_cats = []

missing_cats = books.loc[books["simple_category"].isna(), ["isbn13", "description"]].reset_index(drop=True)

In [22]:
# Get descriptions and ISBNs
sequences = missing_cats["description"].reset_index(drop=True)
isbns = missing_cats["isbn13"].reset_index(drop=True)

# Generate predictions in batches
predicted_cats = generate_predictions(sequences, fiction_categories)

# Keep ISBNs aligned
isbn = list(isbns)

100%|██████████| 88/88 [24:59<00:00, 17.04s/it]


In [23]:
missing_predicted_df = pd.DataFrame({
    "isbn13": isbn,
    "predicted_category": predicted_cats
})

In [24]:
missing_predicted_df

,isbn13,predicted_category
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1399,9788125026600,Nonfiction
1400,9788171565641,Fiction
1401,9788172235222,Fiction
1402,9788173031014,Nonfiction


In [25]:
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")
books["simple_category"] = np.where(books["simple_category"].isna(), books["predicted_category"], books["simple_category"])
books = books.drop(columns=["predicted_category"])

In [26]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title and subtitle,tagged_description,simple_category
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [27]:
books.to_csv('books_categorized.csv', index=False)